In [11]:
#install pylucene from http://lucene.apache.org/pylucene/

import sys
import lucene
import os
from java.io import File
from java.nio.file import Paths
from lucene import JavaError

from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.core import WhitespaceAnalyzer
from org.apache.lucene.document import Document, Field, TextField, FieldType
from org.apache.lucene.search import FuzzyQuery, MultiTermQuery, IndexSearcher
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, DirectoryReader, FieldInfo, IndexOptions,MultiReader, Term
from org.apache.lucene.store import RAMDirectory, SimpleFSDirectory
from org.apache.lucene.analysis.miscellaneous import LimitTokenCountAnalyzer
from org.apache.lucene.search.spans import SpanNearQuery, SpanQuery, SpanTermQuery, SpanMultiTermQueryWrapper
from org.apache.lucene.queryparser.classic import MultiFieldQueryParser, QueryParser

from org.apache.lucene.search.similarities import BM25Similarity
from org.apache.lucene import analysis, document, index, queryparser, search, store, util

In [2]:
lucene.initVM(vmargs=['-Djava.awt.headless=true'])
# storeDir=os.path.dirname(os.path.abspath("/foo"))
# store = SimpleFSDirectory(Paths.get(storeDir))
store = store.RAMDirectory()

In [3]:
#define field settings
t2 = FieldType()
t2.setStored(False)
t2.setTokenized(True)
t2.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS)

analyzer = StandardAnalyzer()
config = IndexWriterConfig(analyzer)
config.setOpenMode(IndexWriterConfig.OpenMode.CREATE)
writer = IndexWriter(store, config)

In [4]:
def addDoc(text, writer):
        doc = Document()
        doc.add(Field("field", text, t2))        
        writer.addDocument(doc)

In [5]:
addDoc("bananas loose", writer)
addDoc("organic bananas with toast", writer)
addDoc("kids bananas bread", writer)
writer.commit()
writer.close()

In [6]:
searcher=IndexSearcher(DirectoryReader.open(store))
searcher.setSimilarity(BM25Similarity())

In [8]:
queryparser.classic.QueryParser( "fieldname", analyzer)
parser = queryparser.classic.QueryParser("field", analyzer)
query = parser.parse("loose bananas for kids")
# query = SpanMultiTermQueryWrapper(FuzzyQuery(query))
hits = searcher.search(query, 3).scoreDocs
print (hits)

[<ScoreDoc: doc=0 score=0.5865056 shardIndex=0>, <ScoreDoc: doc=2 score=0.50652754 shardIndex=0>, <ScoreDoc: doc=1 score=0.053412557 shardIndex=0>]


In [12]:
#query in simple way
# clauses=[1,2]
# clauses[0] =  SpanMultiTermQueryWrapper(FuzzyQuery( Term("field", "bana3as")));
# clauses[1] =  SpanMultiTermQueryWrapper(FuzzyQuery( Term("field", "kids")));
query = FuzzyQuery( Term("field", "bananas"))
hits = searcher.search(query, 3).scoreDocs
print (hits)

JArray<object>[<ScoreDoc: doc=0 score=0.07027968 shardIndex=0>, <ScoreDoc: doc=2 score=0.060696084 shardIndex=0>, <ScoreDoc: doc=1 score=0.053412557 shardIndex=0>]


In [53]:
ocr_query="orgc bananas".split()
for i, val in enumerate(ocr_query):
    ocr_query[i] =  SpanMultiTermQueryWrapper(FuzzyQuery( Term("field", ocr_query[i])));
query = SpanNearQuery(ocr_query,50, False)
hits = searcher.search(query, 1).scoreDocs
print (hits)

JArray<object>[]


In [21]:
QueryParser.escape("if :: f ")

'if \\:\\: f '